# Data Engineer Intern Assessment

## Saurabh Sankhe

## Description about the project 
<b>
    <p>
        In this assignment we are going to visit https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population website and extract the data about each cities which is displayed in the first table of the page. There are links to each city in that page which then gives us extra information about the city and has various other information, we are going to extract first short paragraph about the city i.e City Description and the list of Zipcodes available in that particular city and export the complete data to BigQuery importable csv file.
    </p>
</b>


### Enter the path where you want the csv to be exported

In [68]:
#Path where the csv file should be expoorted
csv_Path="C://Users/Saurabh/Desktop/US_City_Data.csv"

# Steps to upload data to Google Clouds BigQuery DataWarehousing Solution (To be followed after running the jupyter notebook):
1. Run the jupyter notebook to create csv file at desired location
2. Go to Google Cloud's BigQuery page from your account 
3. <b>Create</b> a <b>Project if there is no project</b> in your account else <b>select the project</b> in which you wish to upload the data
4. <b>Create</b> the <b>data-set</b> if there is no data-set in your project else <b>select</b> the <b>data-set</b> under which you want your data 
5. Once the data-set is selected Go-to <b>Create Table</b>
6. In <b>Source</b> Select create table from as - <b>Upload</b>
7. <b>Browse</b> the path where you have exported your <b>CSV file</b>
8. Select <b>File Format</b> as <b>CSV</b>
9. Go-to <b>Table Name</b> and <b>Give the Table the desired name</b>. Preferable Name: US_City_Data
10. Go-to <b>Schema</b> => And <b>select the Checkbox under Auto-detect Schema and input parameters</b> 
        


## WebScraping the Wikipedia website

<b><p>In order to extract the data grom the given wikipedia webpage we are going to use BeautifulSoup and requests from the python libraries which will fetch the relevant data that will eventually be converted into a pandas dataframe. Thus, we are going to use the following libraries.</p>
<font color=red>
1. requests - to get webcontent from the weburl
2. BeautifulSoup - to properly arrange the data as per tags to which we can extract the relevant fields as we want
3. Pandas - to convert the data into dataframe which will eventually be written to .csv file
4. re - to apply regular expression
</font>
</b>

In [45]:
#Importing the required libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [46]:
#Storing the url string into a variable
url="https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"

In [47]:
#stores all the url object to web_url variable
web_url = requests.get(url)

In [48]:
#Changing the content of web_url using lxml/html parser to modify the data which could be used to extract data in specific manner
soup=BeautifulSoup(web_url.content,'lxml')

<b>
<font color=black>
    After inspecting the webpage of wikipedia for which we are scraping the data we get to know that there are 4 tables in the webpage and all the tables have class: 'wikitable sortable'.
</font>
<br><br>
<font color=black>
    Since, we are interested in the first table of the source page going forward we will extract the contents of first table which is the relevant and important information that is needed to be converted and exported into .csv file
</font>
</b>

In [49]:
#Extracting all the tables with class:wikitable sortable
table_data=soup.find_all('table',{'class':'wikitable sortable'})

In [50]:
#Checking the lengthof the table data 
len(table_data)

4

<b><font color='black'>From the above length we can say that parser has extracted 4 tables and that all 4 tables have the same class of wikitable sortable. We need to extract all the data from table 1 and have our data manupulation on that table. Going forward we have only extracted data from table_data[0] which is the data that we need to extract.</blue></b>

In [51]:
#Creating a list of table_header to store the column names of the data
table_header = []
for th in table_data[0].find_all('th'):
    table_header.append(th.text.replace("\n",""))

<b>Also now, if we look at our original table we have sub columns for "2016 land area" and "2016 population density". So we need to handle our table headers and modify them in such a way that there are equal number of columns and our column headers. Thus we have created seperate columns for land area and population density as per miles and kilometers</b>

In [52]:
#Creating new list to store modified columns
thli=[]
for th in table_header:
    if 'land area' in th:
        thli.append(th+' sq mi')
        thli.append(th+' sq km')
    elif 'population density' in th:
        thli.append(th+' per sq mi')
        thli.append(th+' per sq km')
    else:
        thli.append(th)

#Assigning the newly created list to table_header to match the number of columns each row will have.
table_header = thli

In [53]:
#Printing the list of table_header to console
table_header

['2018rank',
 'City',
 'State[c]',
 '2018estimate',
 '2010Census',
 'Change',
 '2016 land area sq mi',
 '2016 land area sq km',
 '2016 population density per sq mi',
 '2016 population density per sq km',
 'Location']

<b> Extracting all the table rows i.e tr tags from the first table from which we will extract table data for each row and store it in col_data which is a list of individual row data and maintain a list for each row in row_data. Thus, our data is structured in following way:</b>

<b><font color=red> row_data => [row1, row2, row3.....] </font></b>

In [54]:
#Extracting all the data inside each tr tag for 1st table and storing it in tr_data
tr_data=table_data[0].find_all('tr')

In [55]:
#Creating list of row_data which will contain list for individual row as explained above. 
#col_data contains data for particular row which is then appended to row_data.
row_data=[]
for tr in tr_data[1:]:
    td_data=tr.find_all('td')
    col_data=[]
    for td in td_data:
        col_data.append(td.text.replace("\n",""))
    row_data.append(col_data)

In [56]:
#Creating a dataframe from the extracted data
df=pd.DataFrame(row_data,columns=table_header)

In [57]:
#Printing the shape of the dataframe to console
df.shape

(314, 11)

In [58]:
#Printing the head of the dataframe to console
df.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area sq mi,2016 land area sq km,2016 population density per sq mi,2016 population density per sq km,Location
0,1,New York City[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


## Performing WebScraping for each City page
### Extracting links from the city column

<b> Let us first see how our links are before we perform webscraping to all the links. After looking at the webpage we get to know that there are links to each city in the <font color='red'>City</font> column which we will extract and look how the link is to <font color='red'>NewYork City</font> (in our case).</b>

In [59]:
#Extracting the link to NewYork City
tr_data[1].find_all('td')[1].find('a').get('href')

'/wiki/New_York_City'

<b>Since, we just have the the naked url we need to append https://en.wikipedia.org to each url and create a list of url names which are stored in <font color='red'>links_of_cities</font></b>

In [60]:
#Finding all the links to cities and appending the above mentioned wikipedia URL to create a valid url.
#Storing the valid URLs to list links_of_cities
links_of_cities=[]
for tr in tr_data[1:]:
    links_of_cities.append('https://en.wikipedia.org'+tr.find_all('td')[1].find('a').get('href'))

In [61]:
#Printing the list of urls to console
links_of_cities

['https://en.wikipedia.org/wiki/New_York_City',
 'https://en.wikipedia.org/wiki/Los_Angeles',
 'https://en.wikipedia.org/wiki/Chicago',
 'https://en.wikipedia.org/wiki/Houston',
 'https://en.wikipedia.org/wiki/Phoenix,_Arizona',
 'https://en.wikipedia.org/wiki/Philadelphia',
 'https://en.wikipedia.org/wiki/San_Antonio',
 'https://en.wikipedia.org/wiki/San_Diego',
 'https://en.wikipedia.org/wiki/Dallas',
 'https://en.wikipedia.org/wiki/San_Jose,_California',
 'https://en.wikipedia.org/wiki/Austin,_Texas',
 'https://en.wikipedia.org/wiki/Jacksonville,_Florida',
 'https://en.wikipedia.org/wiki/Fort_Worth,_Texas',
 'https://en.wikipedia.org/wiki/Columbus,_Ohio',
 'https://en.wikipedia.org/wiki/San_Francisco',
 'https://en.wikipedia.org/wiki/Charlotte,_North_Carolina',
 'https://en.wikipedia.org/wiki/Indianapolis',
 'https://en.wikipedia.org/wiki/Seattle',
 'https://en.wikipedia.org/wiki/Denver',
 'https://en.wikipedia.org/wiki/Washington,_D.C.',
 'https://en.wikipedia.org/wiki/Boston',
 'h

<b>Now since we have valid URLs to each city we will visit the city page and then extract a short description of cities and the zipcodes that belong to those cities</b>

<b><font color=red>Note: If the city have no zipcodes we are going to append null string to the list. Also we clean the data by eleminating \n in the text which we are going to extract</font></b>

In [62]:
#Creating two new lists to store short description of city and zipcodes
#short_desc_of_city will store a short description to each city
#zip_codes will store zipcodes we get in that particular city
short_desc_of_city=[]
zip_codes=[]
for city in links_of_cities:
    sp=BeautifulSoup(requests.get(city).content,'lxml')
    if sp.find_all('p')[1].text!="\n":
        short_desc_of_city.append(sp.find_all('p')[1].text.replace("\n",""))
    else:
        short_desc_of_city.append(sp.find_all('p')[2].text.replace("\n",""))
    if len(sp.find_all('div',{'class':'postal-code'}))>0:
        zip_codes.append(sp.find_all('div',{'class':'postal-code'})[0].text.replace("\n",""))
    else:
        zip_codes.append("")

<b> Now, Since we have Short Description and the Zipcodes of that city we will create a new column for storing those two data in our original dataframe. Going forward we will clean the zip-codes and Short_Description_of_City columns

In [63]:
# Creating two new columns in our original dataframe to store short decription and the zipcodes of the city
df['Short_Description_of_City']=short_desc_of_city
df['zip-codes']=zip_codes

### Cleaning the Zip-Codes and Short_Description_of_City column to remove unwanted data

In [64]:
#Cleaning the zip-codes column
df['zip-codes']=df['zip-codes'].apply(lambda x:"".join(re.findall("[0-9 x+,–]",re.sub("[\(\[].*?[\)\]]", "", x))))

In [65]:
#Cleaning the Short_Description_of_City
df['Short_Description_of_City']=df['Short_Description_of_City'].apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))

In [66]:
#Printing the head of the dataframe to console
df.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area sq mi,2016 land area sq km,2016 population density per sq mi,2016 population density per sq km,Location,Short_Description_of_City,zip-codes
0,1,New York City[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,"The City of New York, usually called either Ne...","100xx–104xx, 11004–05, 111xx–114xx, 116xx"
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,"Los Angeles ; Spanish: Los Ángeles), officiall...","90001–90084, 90086–90089, 90091, 90093–90097, ..."
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,"Chicago , locally also /ʃɪˈkɔːɡoʊ/), officiall...","606xx, 607xx, 608xx"
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,Houston HEW-stən) is the most populous city i...,"770xx, 772xx"
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,Phoenix is the capital and most populous city...,85001–85099


## Exporting the data to BigQuery Importable CSV File

<b>Since BigQuery expects the data to be encoded in UTF-8 format. We are going to export the data in the same format.</b>

<b><font color=red>NOTE: Schema will be auto-detected while using UTF-8 encoding. Thus, the csv exported will be ready to be imported in BigQuery</font></b>


In [67]:
df.to_csv(csv_Path,index=False,encoding='UTF-8')